# REPORTES DE EJECUCION DE OBRAS

La finalidad de los presentes reportes es exponer la ejecución presupuestaria de obras (partidas 421 y 422) y, más específcamente, la ejecución de Módulos Básicos (Programa 29). La información contenida en los mismos será útil para tareas de control y, especialmente, para la elaboración de los Estados Contables

## Datos iniciales

El único dato que debería modificarse es el relativo al ejercicio sobre el cual se pretende trabajar.

In [ ]:
from invicoctrlpy.gastos.ejecucion_obras.ejecucion_obras import EjecucionObras

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment',None)

import itables
itables.options.lengthMenu = [10, 50, 100, -1]
itables.options.maxBytes = 0

from  pivottablejs import pivot_ui

EJERCICIO = '2023'
UPDATE_DB = False
ejecucion_obras = EjecucionObras(ejercicio=EJERCICIO, update_db=UPDATE_DB)

## Reportes

### Detalle Obras Ejecutadas en el ejercicio seleccionado (SIIF)

La finalidad del presente reporte es exponer la ejecución presupuestaria de obras (partidas 421 y 422) con datos neteamente sacados del SIIF (rf602 + rf610) para el ejercicio seleccionado. 

In [ ]:
siif_ejec_obras_actual = ejecucion_obras.reporte_siif_ejec_obras_actual()
itables.show(siif_ejec_obras_actual, column_filters="footer", dom="lrtip")

### Planillómetro (Icaro + Descripciones SIIF)

La finalidad del presente reporte es exponer la ejecución presupuestaria de obras (partidas 421 y 422) con datos de Icaro y descripcion del SIIF (también es posible utilizar las denominaciones de las estructuras presupuestias directamente de Icaro) pivoteado por ejercicio. Además, las obras se desegregan en Obras en Curso, Terminadas Anterior y Terminadas en el presente Ejercicio (una vez que la obra llega al 100% de avance, toda la ejecución posterior, si la hubiera, es considerada como terminada independientemente del porcentaje de avance)

In [ ]:
planillometro = ejecucion_obras.reporte_planillometro(desagregar_fuente=False)
itables.show(planillometro, column_filters="footer", dom="lrtip")

### Módulos Básicos por Convenio (Icaro + Descripciones SIIF)

La finalidad del presente reporte es exponer la ejecución presupuestaria de Módulos Básicos (partidas 421 y 422 en Programa 29) con datos de Icaro y descripcion del SIIF (también es posible utilizar las denominaciones de las estructuras presupuestias directamente de Icaro) pivoteado por convenio (en este caso solo se incluye fuente 11 pero, si se desea pivotear por ejercicio, se incluyen todas las fuentes). Además, las obras se desegregan en Obras en Curso, Terminadas Anterior y Terminadas en el presente Ejercicio (una vez que la obra llega al 100% de avance, toda la ejecución posterior, si la hubiera, es considerada como terminada independientemente del porcentaje de avance)

In [ ]:
icaro_mod_basicos_convenios = ejecucion_obras.reporte_icaro_mod_basicos()
itables.show(icaro_mod_basicos_convenios, column_filters="footer", dom="lrtip")

## Tablas Auxiliares

### Ejecucion SIIF con Descripcion hasta el ejercicio seleccionado

In [ ]:
siif_ejec_obras = ejecucion_obras.import_siif_obras_desc()
itables.show(siif_ejec_obras, column_filters="footer", dom="lrtip")

### Ejecución ICARO con Descripción hasta el Ejercicio seleccionado

In [ ]:
icaro_carga_desc_siif = ejecucion_obras.import_icaro_carga_desc(es_desc_siif=False)
itables.show(icaro_carga_desc_siif, column_filters="footer", dom="lrtip")

### Ejecución ICARO Módulos Básicos con Descripción hasta el Ejercicio seleccionado

In [ ]:
icaro_mod_bas = ejecucion_obras.import_icaro_mod_basicos(es_desc_siif=True)
itables.show(icaro_mod_bas, column_filters="footer", dom="lrtip")

## EXTRA:

### Excel

In [ ]:
with pd.ExcelWriter('Planillometro al ' + EJERCICIO + '.xlsx') as writer:
    siif_ejec_obras_actual.to_excel(writer, sheet_name='EjecuccionSIIF' + EJERCICIO, index=False)
    siif_desc_icaro_actual = ejecucion_obras.import_siif_obras_desc_icaro()
    siif_desc_icaro_actual.to_excel(writer, sheet_name='EjecuccionSIIFDescICARO' + EJERCICIO, index=False)
    planillometro.to_excel(writer, sheet_name='Planillometro', index=False)
    planillometro_full_icaro = ejecucion_obras.reporte_planillometro(full_icaro=True, es_desc_siif=False)
    planillometro_full_icaro.to_excel(writer, sheet_name='PlanillometroFullIcaro', index=False)

with pd.ExcelWriter('Módulos Básicos Ejecutados Hasta el ' + EJERCICIO + '.xlsx') as writer:
    siif_ejec_mod_basicos = siif_ejec_obras_actual.copy()
    siif_ejec_mod_basicos = siif_ejec_mod_basicos.loc[siif_ejec_mod_basicos['estructura'].str.startswith('29')]
    siif_ejec_mod_basicos.to_excel(writer, sheet_name='EjecuccionSIIF' + EJERCICIO, index=False)
    icaro_mod_basicos_convenios.to_excel(writer, sheet_name='ModBasicosPorConvFte11', index=False)
    icaro_mod_basicos_ejercicios = ejecucion_obras.reporte_icaro_mod_basicos(por_convenio=False)
    icaro_mod_basicos_ejercicios.to_excel(writer, sheet_name='ModBasicosPorEjercicio', index=False)